In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from moviepy.editor import VideoFileClip
global attempted
global left_line,right_line,prev_pts
%matplotlib inline

In [32]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # No. of values
        self.nvalues = 3
        # Current iteration
        self.current = 0 
        # Array of last self.nvalues worth of polynomial coefficients
        self.last_fit = np.empty((self.nvalues,3,))
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        
global left_line,right_line,prev_pts
left_line = Line()
right_line = Line()
prev_pts = np.int32([[ 155  ,648],[ 549 , 450],[1216,  648],[ 769,  450]])

In [3]:
images = glob.glob("camera_cal/calibration*.jpg")
objpoints = []
imgpoints = []

objp = np.zeros((9*6,3),np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

for imgname in images:
    img = mpimg.imread(imgname)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
#     plt.imshow(gray,cmap='gray')

    ret, corners = cv2.findChessboardCorners(gray,(9,6),None)
    if ret==True:
        objpoints.append(objp)
        imgpoints.append(corners)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


In [4]:
def color_n_gradient(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    image = cv2.undistort(img,mtx, dist, None, mtx)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]
    h_channel = hls[:,:,0]

    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = sx_thresh[0]
    thresh_max = sx_thresh[1]
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = s_thresh[0]
    s_thresh_max = s_thresh[1]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(s_binary), sxbinary, s_binary))
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary

In [5]:
def process_image(img):
    global prev_pts
    image = np.copy(img)
    imshape = image.shape
    offset = int(imshape[0]*0.1)
    hsv_bit = color_n_gradient(image)
    
    edge = np.zeros_like(hsv_bit)
    mask_bit_value = 255
    mask = np.array([[(0,imshape[0]),(imshape[1]*0.53, imshape[0]/1.8), 
                      (imshape[1]*0.47, imshape[0]/1.8), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(edge,mask,mask_bit_value)
    
    hsv_bit = cv2.bitwise_and(hsv_bit,edge)
#     plt.imshow(hsv_bit,cmap="gray")
    
    row = 2
    theta = np.pi/90
    threshold = 55
    min_line_len = 40
    max_line_gap = 15

    lines = cv2.HoughLinesP(hsv_bit,row,theta,threshold,min_line_len,max_line_gap)
#     print(lines.shape)
    if(lines==None):
        return

    xa = []
    xb = []
    xc = []
    xd = []
    ya = int(image.shape[0])
    yb = int(image.shape[0]/1.6)

    for line in lines:
        for x1,y1,x2,y2 in line:
#             cv2.line(image,(x1,y1),(x2,y2),(255,0,0),2)
            m = (y2-y1)/(x2-x1)
            c = y2-(m*x2)
            if(-0.9<m<-0.5):
                xa.append(int((ya-c)/m))
                xb.append(int((yb-c)/m))

            elif(0.9>m>0.55):
                xc.append(int((ya-c)/m))
                xd.append(int((yb-c)/m))
    try:
        x1 = int(np.mean(xa))
    except:
        x1 = prev_pts[0,0]+offset
    try:
        x2 = int(np.mean(xb))
    except:
        x2 = prev_pts[1,0]+offset
    try:
        x3 = int(np.mean(xc))
    except:
        x3 = prev_pts[2,0]-offset
    try:
        x4 = int(np.mean(xd))
    except:
        x4 = prev_pts[3,0]-offset
    cv2.line(image,(x1,ya),(x2,yb),(0,255,0),8)
    cv2.line(image,(x3,ya),(x4,yb),(0,255,0),8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image,'Left Slope = '+str((yb-ya)/(x2-x1)),(100,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(image,'Right Curvature = '+str((yb-ya)/(x4-x3)),(100,80), font, 1,(255,255,255),2,cv2.LINE_AA)
    
    ya = int(image.shape[0]*0.9)
    prev_pts = np.int32([[x1-offset,ya],[x2-offset,yb],[x3+offset,ya],[x4+offset,yb]])
    return prev_pts
#     return image

In [6]:
def get_warped(cng, src):
    dst = np.float32([[-200,0],[1430,0],[0,720],[1280,720]])
    idx=[1,3,0,2]
    
    M = cv2.getPerspectiveTransform(np.float32(src[idx]),dst)
    Minv = cv2.getPerspectiveTransform(dst, np.float32(src[idx]))

    binary_warped = cv2.warpPerspective(cng,M,(1280,720),flags=cv2.INTER_LINEAR)
    
    return binary_warped,Minv

In [100]:
def line_detect(binary_warped):
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
#     print(leftx.shape)
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
#     print(left_fit.shape)
    return left_fit,right_fit
#     return show_finished(left_fit,right_fit,binary_warped,Minv)

In [8]:
def line_detect_after(binary_warped,left_fit,right_fit):
    global lf,rf
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    lold,rold = cal_curve_real(left_fit,right_fit,binary_warped)
    lnew,rnew = cal_curve_real(left_fit,right_fit,binary_warped)
    if(abs(lnew-lold)>500 or abs(rnew-rold)>500):
        left_fit = lf
        right_fit = rf
    return left_fit,right_fit

In [15]:
def cal_curve_real(left_fit,right_fit,binary_warped):
    # Define conversions in x and y from pixels space to meters
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    leftx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    rightx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    return(left_curverad,right_curverad)
    # Example values: 632.1 m    626.2 m

In [25]:
def cal_center_distance(left_fit,right_fit):
    l_base = left_fit[0]*720**2 + left_fit[1]*720 + left_fit[2]
    r_base = right_fit[0]*720**2 + right_fit[1]*720 + right_fit[2]
    
    x_center = (l_base + r_base)/2
    img_center = 1280/2 
    diff = x_center - img_center
    xm_per_pix = 3.7/700
    if(diff<0):
        state = 'right'
    elif(diff>0):
        state = 'left'
    elif(diff == 0):
        state = 'center'
    return str(diff*xm_per_pix)+' to the ' + state

In [9]:
def sanity_check(left_fit,right_fit,binary_warped):
    l,r = cal_curve_real(left_fit,right_fit,binary_warped)
    if((l/r>3.0 or r/l>3.0) and l<=10000 and r<=10000):
        print("Failed because L_curvature = {} and R_curvature = {}".format(l,r))
        return False
    
    elif(400>abs(left_fit[2]-right_fit[2]) or abs(left_fit[2]-right_fit[2])>1000):
        print("Failed because lane distance = {}".format(abs(left_fit[2]-right_fit[2])))
        return False    

    return True

In [10]:
def update_line(lf,rf,binary_warped):
    global attempted
    global left_line,right_line
    n = left_line.nvalues
    if(left_line.current == n-1):
        cur = -1
    else:
        cur = left_line.current
        
    
    left_line.last_fit[cur+1] = lf
    right_line.last_fit[cur+1] = rf
    
    if(left_line.best_fit == None):
        left_line.best_fit = lf
        right_line.best_fit = rf
        for x in range(0,n):
            left_line.last_fit[x] = lf
            right_line.last_fit[x] = rf
        
    else:
        
        left_line.best_fit = np.mean(left_line.last_fit,axis=0)
        right_line.best_fit = np.mean(right_line.last_fit,axis=0)

    left_line.radius_of_curvature, right_line.radius_of_curvature = cal_curve_real(left_line.last_fit[cur+1], 
                                                                                   right_line.last_fit[cur+1], binary_warped)

In [11]:
def show_finished(img,left_fit,right_fit,binary_warped,Minv):
    # image = cv2.imread()
    # Create an image to draw the lines on
    ext = 1.2
    ploty = np.linspace(0, binary_warped.shape[0]*ext-1, binary_warped.shape[0]*ext )
#     print(left_fit)
#     print(ploty)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
#     print(left_fit[0]*ploty**2)
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    warp_zero = np.zeros_like(np.empty((binary_warped.shape[0]*ext,binary_warped.shape[1]*ext))).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    return result
#     plt.imshow(result)

In [21]:
def pipeline(im):
    global attempted
    global left_line
    global right_line
    cur = left_line.current
#     img = mpimg.imread("test_images/test6.jpg")
    cng = color_n_gradient(im)
    src = process_image(im)
#     return src
    
    binary_warped,Minv = get_warped(cng,src)
    if(attempted == False):
        lf,rf = line_detect(binary_warped)
        attempted = True
    else:
        lf,rf = line_detect_after(binary_warped,left_line.last_fit[cur],right_line.last_fit[cur])
    
    if(sanity_check(lf,rf,binary_warped) == True):
        update_line(lf,rf,binary_warped)
    else:
        attempted = False
    
    final = show_finished(im, left_line.best_fit, right_line.best_fit, binary_warped, Minv)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(final,'Left Curvature = '+str(left_line.radius_of_curvature),(100,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(final,'Right Curvature = '+str(right_line.radius_of_curvature),(100,80), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(final,'Vehicle is {}'.format(cal_center_distance(left_line.last_fit[cur],right_line.last_fit[cur])),(100,110), font, 1,(255,255,255),2,cv2.LINE_AA)
    if(attempted == False):
        cv2.putText(final,'RESET',(900,50), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,0,0),3,cv2.LINE_AA)

    if(left_line.current == left_line.nvalues-1):
        left_line.current = 0
        right_line.current = 0
    else:
        left_line.current += 1
        right_line.current += 1
                                                                   
    return final

In [33]:
global attempted
attempted = False
clip = VideoFileClip('project_video.mp4') # can be gif or movie
new_clip = clip.fl_image(pipeline)
new_clip.write_videofile("output_crop.mp4", audio=False)

[MoviePy] >>>> Building video output_crop.mp4
[MoviePy] Writing video output_crop.mp4


 24%|██▍       | 307/1261 [01:11<04:15,  3.74it/s]

Failed because L_curvature = 2163.143185178186 and R_curvature = 8126.3442834494135


 25%|██▍       | 311/1261 [01:12<04:25,  3.58it/s]

Failed because L_curvature = 5223.08999217125 and R_curvature = 1545.9927627536513


...

Failed because L_curvature = 7420.496017180957 and R_curvature = 209.84428262782316


100%|█████████▉| 1256/1261 [05:01<00:01,  4.39it/s]

Failed because L_curvature = 7708.111276771923 and R_curvature = 2322.4583483081656


100%|█████████▉| 1260/1261 [05:02<00:00,  4.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_crop.mp4 

